In [1]:
!pip install tensorflow tensorflow-gpu pandas matplotlib sklearn opencv-python-headless >/dev/null 2>&1
!pip install gradio jinja2 >/dev/null 2>&1
!pip install wget >/dev/null 2>&1
!pip install kaggle >/dev/null 2>&1
!pip install gradio torch transformers

print("Done")

Done


In [2]:
import gradio as gr
import torch
from transformers import BertForQuestionAnswering, BertTokenizer, AutoTokenizer, AutoModelForSequenceClassification
import os
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
qa_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
qa_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [4]:
toxicity_tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
toxicity_model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

In [5]:
def answer_question(question, context):
    inputs = qa_tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors='pt')
    model_output = qa_model(**inputs)
    answer_start_scores = model_output.start_logits
    answer_end_scores = model_output.end_logits
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = qa_tokenizer.convert_tokens_to_string(qa_tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    answer = 'KASSbot says: ' + answer.capitalize()
    return answer

In [6]:
def highlight_text(text, words, color):
    for word in words:
        text = re.sub(f'({word})', f'<mark style="background-color: {color}">{word}</mark>', text, flags=re.IGNORECASE)
    return text

In [7]:
def classify_toxicity(text):
    inputs = toxicity_tokenizer.encode_plus(text, return_tensors='pt')
    outputs = toxicity_model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probabilities)
    return predicted_class.item() == 1

In [8]:
def process_text(spam_words, fraud_words, sexual_words, text):
    spam_words = spam_words.lower().split(',')
    fraud_words = fraud_words.lower().split(',')
    sexual_words = sexual_words.lower().split(',')

    spam_words = [word.strip() for word in spam_words]
    fraud_words = [word.strip() for word in fraud_words]
    sexual_words = [word.strip() for word in sexual_words]

    text = highlight_text(text, spam_words, 'lightgreen')
    text = highlight_text(text, fraud_words, 'lightblue')
    text = highlight_text(text, sexual_words, 'lightcoral')

    sentences = sent_tokenize(text)
    for i, sentence in enumerate(sentences):
        if classify_toxicity(sentence):
            sentences[i] = f'<mark style="background-color: red">{sentence}</mark>'
    text = ' '.join(sentences)

    return text

In [9]:
def wrapper(choice, question, context, spam_words, fraud_words, sexual_words, text):
    if choice == 'KASSBot':
        return answer_question(question, context)
    elif choice == 'MarketSense':
        return process_text(spam_words, fraud_words, sexual_words, text)

In [10]:
iface = gr.Interface(fn=wrapper,
                        inputs=[gr.inputs.Dropdown(choices=['KASSBot', 'MarketSense'], label="Choose function"),
                        gr.inputs.Textbox(lines=3, placeholder='Enter your question here...', label="Question for KASSBot"),
                        gr.inputs.Textbox(lines=20, placeholder='Enter the context here...', label="Context for KASSBot"),
                        gr.inputs.Textbox(lines=2, placeholder="Enter Spam Words...", label="Spam Words for MarketSense (Green)"),
                        gr.inputs.Textbox(lines=2, placeholder="Enter Fraud Words...", label="Fraud Words for MarketSense (Blue)"),
                        gr.inputs.Textbox(lines=2, placeholder="Enter Sexual Words...", label="Sexual Words for MarketSense (Red)"),
                        gr.inputs.Textbox(lines=5, placeholder="Enter Text to Analyze...", label="Text to Analyze for MarketSense")],
                    outputs=gr.outputs.HTML())

iface.launch()

/opt/conda/lib/python3.10/site-packages/gradio/inputs.py:219: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/gradio/inputs.py:222: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/opt/conda/lib/python3.10/site-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/opt/conda/lib/python3.10/site-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://57e224232c345f5ff8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
